In [0]:
# European Power Grid Stress Prediction Model
# Predicts grid stress and blackout risk 4 hours in advance
# Data source: ENTSOE Transparency Platform (2023-2025)
# Countries: 26 European nations

import pandas as pd
import numpy as np
from pyspark.sql import functions as F
from pyspark.sql.window import Window
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime, timedelta
import warnings
warnings.filterwarnings('ignore')

# Display settings
pd.set_option('display.max_columns', 100)
pd.set_option('display.max_rows', 100)

# Data catalog
CATALOG = "curlybyte_solutions_rawdata_europe_grid_load"
SCHEMA_GRID = "european_grid_raw__v2"
SCHEMA_WEATHER = "european_weather_raw"

print("European Power Grid Stress Prediction Model")
print("="*60)
print(f"Catalog: {CATALOG}")
print(f"Grid Schema: {SCHEMA_GRID}")
print(f"Weather Schema: {SCHEMA_WEATHER}")
print("\nLibraries loaded successfully")

In [0]:
# Load core datasets from ENTSOE transparency platform
# Grid data: load, forecasts, cross-border flows
# Weather data: temperature, wind, solar radiation

print("Loading datasets...")
print("="*60)

# Load grid tables
load_actual = spark.table(f"{CATALOG}.{SCHEMA_GRID}.load_actual")
load_forecast = spark.table(f"{CATALOG}.{SCHEMA_GRID}.load_forecast")
crossborder = spark.table(f"{CATALOG}.{SCHEMA_GRID}.crossborder_flows")
generation = spark.table(f"{CATALOG}.{SCHEMA_GRID}.generation")

# Load weather
weather = spark.table(f"{CATALOG}.{SCHEMA_WEATHER}.weather_hourly")

# Dataset sizes
print(f"load_actual:      {load_actual.count():>12,} rows")
print(f"load_forecast:    {load_forecast.count():>12,} rows")
print(f"crossborder:      {crossborder.count():>12,} rows")
print(f"generation:       {generation.count():>12,} rows")
print(f"weather:          {weather.count():>12,} rows")

# Temporal coverage
date_range = load_actual.agg(
    F.min("index").alias("start"),
    F.max("index").alias("end")
).collect()[0]

print(f"\nTemporal coverage: {date_range['start']} to {date_range['end']}")

# Countries
countries = [row['country'] for row in load_actual.select("country").distinct().orderBy("country").collect()]
print(f"Countries ({len(countries)}): {', '.join(countries)}")

In [0]:
# Data quality assessment - check for missing values and coverage gaps

print("Data Quality Assessment")
print("="*60)

def check_nulls(df, name):
    """Check null percentage for each column"""
    total = df.count()
    null_info = []
    for col in df.columns:
        null_count = df.filter(F.col(col).isNull()).count()
        if null_count > 0:
            null_info.append((col, null_count, null_count/total*100))
    return null_info

# Check load_actual
print("\n1. LOAD ACTUAL")
nulls = check_nulls(load_actual, "load_actual")
if nulls:
    for col, count, pct in nulls:
        print(f"   {col}: {count:,} nulls ({pct:.2f}%)")
else:
    print("   No missing values")

# Check load_forecast
print("\n2. LOAD FORECAST")
nulls = check_nulls(load_forecast, "load_forecast")
if nulls:
    for col, count, pct in nulls:
        print(f"   {col}: {count:,} nulls ({pct:.2f}%)")
else:
    print("   No missing values")

# Check crossborder
print("\n3. CROSSBORDER FLOWS")
nulls = check_nulls(crossborder, "crossborder")
if nulls:
    for col, count, pct in nulls:
        print(f"   {col}: {count:,} nulls ({pct:.2f}%)")
else:
    print("   No missing values")

# Check data coverage per country
print("\n4. RECORDS PER COUNTRY")
country_counts = load_actual.groupBy("country").count().orderBy("count", ascending=False).collect()
print(f"   {'Country':<10} {'Records':>12}")
print(f"   {'-'*10} {'-'*12}")
for row in country_counts[:10]:
    print(f"   {row['country']:<10} {row['count']:>12,}")
print(f"   ... and {len(country_counts)-10} more countries")

In [0]:
# Build unified dataset by joining load actual, forecast, and crossborder flows
# Join keys: timestamp (index) and country

print("Building Unified Dataset")
print("="*60)

# Step 1: Join load_actual with load_forecast
unified = load_actual.alias("la").join(
    load_forecast.alias("lf"),
    (F.col("la.index") == F.col("lf.index")) & 
    (F.col("la.country") == F.col("lf.country")),
    "inner"
).select(
    F.col("la.index").alias("timestamp"),
    F.col("la.country"),
    F.col("la.Actual_Load").alias("actual_load"),
    F.col("lf.Forecasted_Load").alias("forecast_load")
)

# Step 2: Calculate forecast error (supply-demand mismatch indicator)
unified = unified.withColumn(
    "forecast_error", F.col("actual_load") - F.col("forecast_load")
).withColumn(
    "forecast_error_pct",
    F.coalesce(F.try_divide(F.col("forecast_error"), F.col("forecast_load")) * 100, F.lit(0))
)

print(f"Step 1: Load data joined - {unified.count():,} rows")

# Step 3: Calculate net imports from crossborder flows
imports_df = crossborder.groupBy("index", "to_country").agg(
    F.sum("Value").alias("total_imports")
).withColumnRenamed("to_country", "country")

exports_df = crossborder.groupBy("index", "from_country").agg(
    F.sum("Value").alias("total_exports")
).withColumnRenamed("from_country", "country")

net_flows = imports_df.alias("i").join(
    exports_df.alias("e"),
    (F.col("i.index") == F.col("e.index")) & 
    (F.col("i.country") == F.col("e.country")),
    "outer"
).select(
    F.coalesce(F.col("i.index"), F.col("e.index")).alias("index"),
    F.coalesce(F.col("i.country"), F.col("e.country")).alias("country"),
    F.coalesce(F.col("i.total_imports"), F.lit(0)).alias("total_imports"),
    F.coalesce(F.col("e.total_exports"), F.lit(0)).alias("total_exports")
).withColumn(
    "net_imports", F.col("total_imports") - F.col("total_exports")
)

# Step 4: Join with unified dataset
unified = unified.alias("u").join(
    net_flows.alias("n"),
    (F.col("u.timestamp") == F.col("n.index")) & 
    (F.col("u.country") == F.col("n.country")),
    "left"
).select(
    F.col("u.timestamp"),
    F.col("u.country"),
    F.col("u.actual_load"),
    F.col("u.forecast_load"),
    F.col("u.forecast_error"),
    F.col("u.forecast_error_pct"),
    F.coalesce(F.col("n.total_imports"), F.lit(0)).alias("total_imports"),
    F.coalesce(F.col("n.total_exports"), F.lit(0)).alias("total_exports"),
    F.coalesce(F.col("n.net_imports"), F.lit(0)).alias("net_imports")
)

# Step 5: Calculate import dependency ratio
unified = unified.withColumn(
    "import_ratio",
    F.coalesce(F.try_divide(F.col("net_imports"), F.col("actual_load")), F.lit(0))
)

print(f"Step 2: Crossborder flows joined - {unified.count():,} rows")
print(f"\nColumns: {unified.columns}")

In [0]:
# Integrate weather data by mapping coordinates to countries
# Weather data has lat/lon - we need to aggregate by country

print("Weather Data Integration")
print("="*60)

# Step 1: Get unique coordinates from weather data
print("Step 1: Extracting unique coordinates...")
unique_coords = weather.select("lat", "lon").distinct()
coord_count = unique_coords.count()
print(f"   Unique coordinate pairs: {coord_count:,}")

# Step 2: Map coordinates to countries using reverse geocoding
print("\nStep 2: Mapping coordinates to countries...")

coords_list = unique_coords.collect()
coord_tuples = [(float(row['lat']), float(row['lon'])) for row in coords_list]

# Install and use reverse_geocode
import subprocess
subprocess.check_call(['pip', 'install', 'reverse_geocode', '-q', '--break-system-packages'])
import reverse_geocode

countries_result = reverse_geocode.search(coord_tuples)
mapping_data = [(coord[0], coord[1], loc['country_code']) for coord, loc in zip(coord_tuples, countries_result)]
coord_country_map = spark.createDataFrame(mapping_data, ["lat", "lon", "weather_country"])

print(f"   Coordinates mapped to countries")

# Step 3: Filter to our 26 grid countries
our_countries = ['AT', 'BE', 'BG', 'CH', 'CZ', 'DE', 'DK', 'EE', 'ES', 'FI', 'FR', 'GR', 
                 'HR', 'HU', 'IE', 'IT', 'LT', 'LV', 'NL', 'NO', 'PL', 'PT', 'RO', 'SE', 'SI', 'SK']

weather_countries = [row['weather_country'] for row in coord_country_map.select("weather_country").distinct().collect()]
overlap = set(our_countries) & set(weather_countries)
print(f"   Grid countries with weather data: {len(overlap)}/26")

# Step 4: Join weather with coordinate mapping and aggregate by country/hour
print("\nStep 3: Aggregating weather by country and hour...")

weather_with_country = weather.join(coord_country_map, on=["lat", "lon"], how="inner")
weather_filtered = weather_with_country.filter(F.col("weather_country").isin(our_countries))

weather_agg = weather_filtered.groupBy(
    F.col("weather_country").alias("country"),
    F.date_trunc("hour", F.col("timestamp")).alias("weather_hour")
).agg(
    F.avg("temperature_c").alias("temp_avg"),
    F.min("temperature_c").alias("temp_min"),
    F.max("temperature_c").alias("temp_max"),
    F.stddev("temperature_c").alias("temp_std"),
    F.avg("wind_speed").alias("wind_avg"),
    F.max("wind_speed").alias("wind_max"),
    F.avg("ssrd").alias("solar_radiation_avg")
)

print(f"   Aggregated weather records: {weather_agg.count():,}")

# Step 5: Join weather with unified dataset
print("\nStep 4: Joining weather with grid data...")

unified = unified.withColumn("join_hour", F.date_trunc("hour", F.col("timestamp")))

unified = unified.alias("u").join(
    weather_agg.alias("w"),
    (F.col("u.country") == F.col("w.country")) & 
    (F.col("u.join_hour") == F.col("w.weather_hour")),
    "left"
).select(
    F.col("u.timestamp"),
    F.col("u.country"),
    F.col("u.actual_load"),
    F.col("u.forecast_load"),
    F.col("u.forecast_error"),
    F.col("u.forecast_error_pct"),
    F.col("u.total_imports"),
    F.col("u.total_exports"),
    F.col("u.net_imports"),
    F.col("u.import_ratio"),
    F.col("w.temp_avg"),
    F.col("w.temp_min"),
    F.col("w.temp_max"),
    F.col("w.temp_std"),
    F.col("w.wind_avg"),
    F.col("w.wind_max"),
    F.col("w.solar_radiation_avg")
)

# Verify weather coverage
weather_coverage = unified.filter(F.col("temp_avg").isNotNull()).count() / unified.count() * 100
print(f"   Weather coverage: {weather_coverage:.1f}%")
print(f"\nUnified dataset ready: {unified.count():,} rows, {len(unified.columns)} columns")

In [0]:
# Exploratory Data Analysis
# Analyze distributions, patterns, and relationships in the data

print("Exploratory Data Analysis")
print("="*60)

# Convert sample to Pandas for EDA
sample_fraction = 0.1
eda_sample = unified.sample(fraction=sample_fraction, seed=42).toPandas()
print(f"Sample size for EDA: {len(eda_sample):,} rows ({sample_fraction*100:.0f}% of data)")

# Descriptive statistics
print("\n1. DESCRIPTIVE STATISTICS")
print("-"*60)
numeric_cols = ['actual_load', 'forecast_load', 'forecast_error', 'forecast_error_pct',
                'net_imports', 'import_ratio', 'temp_avg', 'wind_avg']
print(eda_sample[numeric_cols].describe().round(2).to_string())

In [0]:
# Visualize distributions of key variables

fig, axes = plt.subplots(2, 4, figsize=(18, 10))
fig.suptitle('Distribution of Key Variables', fontsize=14, fontweight='bold')

# Row 1: Grid variables
axes[0, 0].hist(eda_sample['actual_load'], bins=50, edgecolor='black', alpha=0.7, color='steelblue')
axes[0, 0].set_xlabel('MW')
axes[0, 0].set_title('Actual Load')

axes[0, 1].hist(eda_sample['forecast_error'].clip(-5000, 5000), bins=50, edgecolor='black', alpha=0.7, color='coral')
axes[0, 1].axvline(x=0, color='red', linestyle='--', linewidth=2)
axes[0, 1].set_xlabel('MW')
axes[0, 1].set_title('Forecast Error (clipped)')

axes[0, 2].hist(eda_sample['forecast_error_pct'].clip(-50, 50), bins=50, edgecolor='black', alpha=0.7, color='green')
axes[0, 2].axvline(x=0, color='red', linestyle='--', linewidth=2)
axes[0, 2].set_xlabel('%')
axes[0, 2].set_title('Forecast Error % (clipped)')

axes[0, 3].hist(eda_sample['import_ratio'].clip(-1, 1), bins=50, edgecolor='black', alpha=0.7, color='purple')
axes[0, 3].axvline(x=0, color='red', linestyle='--', linewidth=2)
axes[0, 3].set_xlabel('Ratio')
axes[0, 3].set_title('Import Ratio (clipped)')

# Row 2: Weather and temporal patterns
axes[1, 0].hist(eda_sample['temp_avg'], bins=50, edgecolor='black', alpha=0.7, color='orangered')
axes[1, 0].set_xlabel('Celsius')
axes[1, 0].set_title('Temperature')

axes[1, 1].hist(eda_sample['wind_avg'], bins=50, edgecolor='black', alpha=0.7, color='teal')
axes[1, 1].set_xlabel('m/s')
axes[1, 1].set_title('Wind Speed')

# Hourly load pattern
eda_sample['hour'] = pd.to_datetime(eda_sample['timestamp']).dt.hour
hourly_load = eda_sample.groupby('hour')['actual_load'].mean()
axes[1, 2].bar(hourly_load.index, hourly_load.values, color='steelblue', edgecolor='black')
axes[1, 2].set_xlabel('Hour of Day')
axes[1, 2].set_ylabel('MW')
axes[1, 2].set_title('Average Load by Hour')

# Load by country (top 10)
country_load = eda_sample.groupby('country')['actual_load'].mean().sort_values(ascending=True).tail(10)
axes[1, 3].barh(country_load.index, country_load.values, color='steelblue', edgecolor='black')
axes[1, 3].set_xlabel('MW')
axes[1, 3].set_title('Average Load by Country (Top 10)')

plt.tight_layout()
plt.show()

print("Key observations:")
print("- Actual load is right-skewed (many small countries, few large)")
print("- Forecast errors are centered near zero with long tails (stress events)")
print("- Import ratio mostly between -25% to +25% (moderate cross-border dependency)")
print("- Clear daily load pattern with morning and evening peaks")

In [0]:
# Correlation analysis between features

print("2. CORRELATION ANALYSIS")
print("-"*60)

# Select numeric columns for correlation
corr_cols = ['actual_load', 'forecast_load', 'forecast_error', 'forecast_error_pct',
             'total_imports', 'total_exports', 'net_imports', 'import_ratio',
             'temp_avg', 'temp_min', 'temp_max', 'wind_avg', 'wind_max', 'solar_radiation_avg']

corr_matrix = eda_sample[corr_cols].corr()

# Plot correlation heatmap
fig, ax = plt.subplots(figsize=(14, 11))
mask = np.triu(np.ones_like(corr_matrix, dtype=bool))
sns.heatmap(corr_matrix, mask=mask, annot=True, fmt='.2f', cmap='RdBu_r',
            center=0, vmin=-1, vmax=1, square=True, linewidths=0.5, ax=ax,
            annot_kws={'size': 9})
ax.set_title('Feature Correlation Matrix', fontsize=14, fontweight='bold')
plt.tight_layout()
plt.show()

# Print strongest correlations
print("\nStrongest correlations (|r| > 0.3):")
print("-"*50)
for i in range(len(corr_matrix.columns)):
    for j in range(i+1, len(corr_matrix.columns)):
        corr_val = corr_matrix.iloc[i, j]
        if abs(corr_val) > 0.3:
            print(f"  {corr_matrix.columns[i]:<20} <-> {corr_matrix.columns[j]:<20}: {corr_val:+.3f}")

In [0]:
# Validate data against the April 28, 2025 Spain/Portugal blackout
# This real event helps us understand what stress patterns look like

print("3. BLACKOUT EVENT ANALYSIS")
print("-"*60)
print("April 28, 2025: Major blackout affected Spain and Portugal")
print("Blackout started approximately 10:45 local time\n")

# Filter for ES and PT on blackout day
blackout_day = unified.filter(
    (F.col("country").isin(["ES", "PT"])) &
    (F.to_date("timestamp") == "2025-04-28")
).orderBy("timestamp", "country").toPandas()

blackout_day['hour'] = pd.to_datetime(blackout_day['timestamp']).dt.hour

# Compare blackout day to normal day (one week before)
normal_day = unified.filter(
    (F.col("country").isin(["ES", "PT"])) &
    (F.to_date("timestamp") == "2025-04-21")
).orderBy("timestamp", "country").toPandas()

normal_day['hour'] = pd.to_datetime(normal_day['timestamp']).dt.hour

# Aggregate by hour for comparison
blackout_hourly = blackout_day.groupby(['country', 'hour'])['actual_load'].mean().reset_index()
normal_hourly = normal_day.groupby(['country', 'hour'])['actual_load'].mean().reset_index()

# Plot comparison
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

for idx, country in enumerate(['ES', 'PT']):
    ax = axes[idx]
    normal = normal_hourly[normal_hourly['country'] == country]
    blackout = blackout_hourly[blackout_hourly['country'] == country]
    
    ax.plot(normal['hour'], normal['actual_load'], 'b-o', label='Normal Day (Apr 21)', linewidth=2, markersize=4)
    ax.plot(blackout['hour'], blackout['actual_load'], 'r-o', label='Blackout Day (Apr 28)', linewidth=2, markersize=4)
    ax.axvline(x=11, color='red', linestyle='--', alpha=0.7, label='Blackout ~11:00')
    ax.set_xlabel('Hour of Day')
    ax.set_ylabel('Load (MW)')
    ax.set_title(f'{country} - Normal vs Blackout Day')
    ax.legend()
    ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

# Show the dramatic load drop
print("Load comparison at key hours:")
print("-"*50)
for country in ['ES', 'PT']:
    normal_10 = normal_hourly[(normal_hourly['country']==country) & (normal_hourly['hour']==10)]['actual_load'].values[0]
    blackout_11 = blackout_hourly[(blackout_hourly['country']==country) & (blackout_hourly['hour']==11)]['actual_load'].values[0]
    drop_pct = (1 - blackout_11/normal_10) * 100
    print(f"  {country}: Normal 10:00 = {normal_10:,.0f} MW, Blackout 11:00 = {blackout_11:,.0f} MW (drop: {drop_pct:.0f}%)")

In [0]:
# Define stress score based on grid conditions
# This will be used to create our prediction target

print("4. TARGET VARIABLE DEFINITION")
print("-"*60)
print("""
Strategy: Create a stress score based on observable grid conditions,
then define a forward-looking target to predict stress BEFORE it happens.

Stress indicators:
- Forecast error: Supply-demand mismatch (primary signal)
- Import ratio: Cross-border dependency (vulnerability)
- Unusual load patterns: Deviation from country norms
""")

# Calculate country-specific baselines for normalization
country_stats = unified.groupBy("country").agg(
    F.avg("forecast_error").alias("country_fe_mean"),
    F.stddev("forecast_error").alias("country_fe_std"),
    F.avg("import_ratio").alias("country_ir_mean"),
    F.stddev("import_ratio").alias("country_ir_std"),
    F.avg("actual_load").alias("country_load_mean"),
    F.stddev("actual_load").alias("country_load_std")
)

# Join country stats and calculate z-scores
unified = unified.join(country_stats, on="country", how="left")

unified = unified.withColumn(
    "forecast_error_zscore",
    (F.col("forecast_error") - F.col("country_fe_mean")) / (F.col("country_fe_std") + 1e-6)
).withColumn(
    "import_ratio_zscore",
    (F.col("import_ratio") - F.col("country_ir_mean")) / (F.col("country_ir_std") + 1e-6)
).withColumn(
    "load_zscore",
    (F.col("actual_load") - F.col("country_load_mean")) / (F.col("country_load_std") + 1e-6)
)

# Calculate stress score (weighted combination)
# Higher absolute z-scores = more unusual/stressed conditions
unified = unified.withColumn(
    "stress_score",
    0.50 * F.abs(F.col("forecast_error_zscore")) +
    0.30 * F.when(F.col("import_ratio_zscore") > 0, F.col("import_ratio_zscore")).otherwise(0) +
    0.20 * F.abs(F.col("load_zscore"))
)

# Define stress thresholds based on percentiles
stress_percentiles = unified.approxQuantile("stress_score", [0.85, 0.95], 0.01)
p85, p95 = stress_percentiles

print(f"Stress score percentiles:")
print(f"  85th percentile: {p85:.3f}")
print(f"  95th percentile: {p95:.3f}")

# Create binary stress indicator (top 15% = high stress)
unified = unified.withColumn(
    "high_stress",
    F.when(F.col("stress_score") >= p85, 1).otherwise(0)
)

# Check distribution
stress_dist = unified.groupBy("high_stress").count().collect()
total = sum([r['count'] for r in stress_dist])
print(f"\nStress distribution:")
for r in stress_dist:
    pct = r['count']/total*100
    label = "High stress" if r['high_stress']==1 else "Normal"
    print(f"  {label}: {r['count']:,} ({pct:.1f}%)")

In [0]:
# Create the PREDICTIVE target: Will high stress occur in the next 4 hours?
# This is the key difference from detection - we predict BEFORE it happens

print("5. FORWARD-LOOKING PREDICTION TARGET")
print("-"*60)
print("""
Goal: Predict if high stress will occur in the NEXT 4 hours
      using only information available NOW.

This gives grid operators lead time to take preventive action.

Prediction horizon: 4 hours = 16 time steps (15-min intervals)
""")

# Define forward-looking window
PREDICTION_HORIZON = 16  # 4 hours ahead (16 x 15-min intervals)

forward_window = Window.partitionBy("country").orderBy("timestamp").rowsBetween(1, PREDICTION_HORIZON)

# Create forward-looking target
# If ANY of the next 16 periods has high_stress=1, then target=1
unified = unified.withColumn(
    "stress_next_4h",
    F.when(F.max("high_stress").over(forward_window) == 1, 1).otherwise(0)
)

# Check new target distribution
target_dist = unified.groupBy("stress_next_4h").count().orderBy("stress_next_4h").collect()
total = sum([r['count'] for r in target_dist])

print("Forward-looking target distribution:")
print("-"*50)
for r in target_dist:
    pct = r['count']/total*100
    label = "Stress coming in 4h" if r['stress_next_4h']==1 else "No stress in 4h"
    print(f"  {r['stress_next_4h']} ({label}): {r['count']:,} ({pct:.1f}%)")

# Validate: Check blackout day - did we have warning signs?
print("\n" + "-"*50)
print("Validation: April 28, 2025 blackout predictions")
print("-"*50)

blackout_check = unified.filter(
    (F.col("country") == "ES") &
    (F.to_date("timestamp") == "2025-04-28") &
    (F.hour("timestamp") >= 6) &
    (F.hour("timestamp") <= 12)
).select(
    "timestamp", "actual_load", "stress_score", "high_stress", "stress_next_4h"
).orderBy("timestamp").toPandas()

print("\nSpain - Morning of blackout (06:00-12:00):")
print(blackout_check.to_string(index=False))

In [0]:
# Feature Engineering
# Create features that capture patterns BEFORE stress events occur

print("6. FEATURE ENGINEERING")
print("-"*60)
print("""
Creating predictive features using only past/current information:
- Temporal features (hour, day, month, cyclical encoding)
- Lag features (past values at t-1, t-4, t-16, t-96)
- Rolling statistics (trends over 4h and 24h windows)
- Rate of change (momentum indicators)
- Weather features and their lags
""")

# Temporal features
import math

unified = unified.withColumn("hour", F.hour("timestamp")) \
    .withColumn("day_of_week", F.dayofweek("timestamp")) \
    .withColumn("month", F.month("timestamp")) \
    .withColumn("is_weekend", F.when(F.dayofweek("timestamp").isin([1, 7]), 1).otherwise(0)) \
    .withColumn("hour_sin", F.sin(2 * math.pi * F.col("hour") / 24)) \
    .withColumn("hour_cos", F.cos(2 * math.pi * F.col("hour") / 24)) \
    .withColumn("month_sin", F.sin(2 * math.pi * F.col("month") / 12)) \
    .withColumn("month_cos", F.cos(2 * math.pi * F.col("month") / 12))

print("Temporal features added: hour, day_of_week, month, is_weekend, cyclical encodings")

# Define windows for lag and rolling features
country_window = Window.partitionBy("country").orderBy("timestamp")
roll_4h = Window.partitionBy("country").orderBy("timestamp").rowsBetween(-15, 0)
roll_24h = Window.partitionBy("country").orderBy("timestamp").rowsBetween(-95, 0)

# Lag features - past values at different horizons
# t-1 (15 min), t-4 (1 hour), t-16 (4 hours), t-96 (24 hours)
lag_configs = [
    ("forecast_error", [1, 4, 16, 96]),
    ("forecast_error_pct", [1, 4]),
    ("import_ratio", [1, 4, 96]),
    ("actual_load", [1, 96]),
    ("temp_avg", [4, 96]),
    ("wind_avg", [4])
]

for col, lags in lag_configs:
    for lag in lags:
        unified = unified.withColumn(f"{col}_lag_{lag}", F.lag(col, lag).over(country_window))

print("Lag features added: forecast_error, import_ratio, load, temp, wind at various horizons")

# Rolling statistics - capture recent trends
unified = unified.withColumn("fe_roll_4h_mean", F.avg("forecast_error").over(roll_4h)) \
    .withColumn("fe_roll_4h_std", F.stddev("forecast_error").over(roll_4h)) \
    .withColumn("fe_roll_24h_mean", F.avg("forecast_error").over(roll_24h)) \
    .withColumn("fe_roll_24h_std", F.stddev("forecast_error").over(roll_24h)) \
    .withColumn("ir_roll_4h_mean", F.avg("import_ratio").over(roll_4h)) \
    .withColumn("ir_roll_24h_mean", F.avg("import_ratio").over(roll_24h)) \
    .withColumn("load_roll_24h_mean", F.avg("actual_load").over(roll_24h)) \
    .withColumn("load_roll_24h_std", F.stddev("actual_load").over(roll_24h)) \
    .withColumn("temp_roll_24h_mean", F.avg("temp_avg").over(roll_24h)) \
    .withColumn("wind_roll_24h_mean", F.avg("wind_avg").over(roll_24h))

print("Rolling statistics added: 4h and 24h means/stds for key variables")

# Rate of change - momentum indicators
unified = unified.withColumn(
    "fe_change_1h", F.col("forecast_error") - F.col("forecast_error_lag_4")
).withColumn(
    "load_change_1h", F.col("actual_load") - F.col("actual_load_lag_1")
).withColumn(
    "load_change_pct", 
    F.coalesce(F.try_divide(F.col("load_change_1h"), F.col("actual_load_lag_1")) * 100, F.lit(0))
).withColumn(
    "temp_change_24h", F.col("temp_avg") - F.col("temp_avg_lag_96")
).withColumn(
    "fe_vs_roll_4h", F.col("forecast_error") - F.col("fe_roll_4h_mean")
)

print("Rate of change features added: hourly and daily momentum indicators")

print(f"\nTotal columns after feature engineering: {len(unified.columns)}")

In [0]:
# Correlation analysis for predictive features
# Important: Check for multicollinearity and identify strongest predictors

print("7. FEATURE CORRELATION ANALYSIS")
print("-"*60)

# Sample for correlation analysis
corr_sample = unified.select(
    # Key predictive features
    'forecast_error', 'forecast_error_pct', 'import_ratio', 'actual_load',
    'temp_avg', 'wind_avg', 'solar_radiation_avg',
    # Lag features
    'forecast_error_lag_1', 'forecast_error_lag_4', 'forecast_error_lag_96',
    'import_ratio_lag_1', 'import_ratio_lag_96',
    # Rolling features
    'fe_roll_4h_mean', 'fe_roll_24h_mean', 'ir_roll_4h_mean',
    'load_roll_24h_mean', 'temp_roll_24h_mean',
    # Rate of change
    'fe_change_1h', 'load_change_pct', 'fe_vs_roll_4h',
    # Target
    'stress_next_4h'
).sample(fraction=0.1, seed=42).toPandas()

# Correlation with target
target_corr = corr_sample.corr()['stress_next_4h'].drop('stress_next_4h').sort_values(key=abs, ascending=False)

print("Correlation with target (stress_next_4h):")
print("-"*50)
for feat, corr in target_corr.items():
    bar = "+" * int(abs(corr) * 30) if corr > 0 else "-" * int(abs(corr) * 30)
    print(f"  {feat:<25} {corr:+.3f} {bar}")

# Plot correlation heatmap for key features
fig, axes = plt.subplots(1, 2, figsize=(16, 6))

# Correlation with target bar chart
ax1 = axes[0]
colors = ['green' if x > 0 else 'red' for x in target_corr.values]
ax1.barh(range(len(target_corr)), target_corr.values, color=colors, alpha=0.7)
ax1.set_yticks(range(len(target_corr)))
ax1.set_yticklabels(target_corr.index)
ax1.set_xlabel('Correlation with stress_next_4h')
ax1.set_title('Feature Correlation with Target')
ax1.axvline(x=0, color='black', linestyle='-', linewidth=0.5)
ax1.invert_yaxis()

# Feature correlation heatmap (subset)
key_features = ['forecast_error', 'forecast_error_pct', 'import_ratio', 
                'fe_roll_4h_mean', 'fe_roll_24h_mean', 'ir_roll_4h_mean',
                'fe_change_1h', 'fe_vs_roll_4h', 'temp_avg', 'wind_avg', 'stress_next_4h']
corr_matrix = corr_sample[key_features].corr()

ax2 = axes[1]
sns.heatmap(corr_matrix, annot=True, fmt='.2f', cmap='RdBu_r', center=0,
            vmin=-1, vmax=1, square=True, linewidths=0.5, ax=ax2, annot_kws={'size': 8})
ax2.set_title('Key Feature Correlation Matrix')

plt.tight_layout()
plt.show()

print("\nKey observations:")
print("- Forecast error features show strongest correlation with future stress")
print("- Rolling means capture trend information useful for prediction")
print("- Some multicollinearity exists (e.g., fe_roll_4h_mean and fe_roll_24h_mean)")

In [0]:
# Prepare final feature set for modeling

print("8. PREPARE MODELING DATASET")
print("-"*60)

# Define feature columns - NO target leakage
feature_cols = [
    # Base grid features
    'actual_load', 'forecast_load', 'forecast_error', 'forecast_error_pct',
    'total_imports', 'total_exports', 'net_imports', 'import_ratio',
    
    # Weather features
    'temp_avg', 'temp_min', 'temp_max', 'wind_avg', 'wind_max', 'solar_radiation_avg',
    
    # Temporal features
    'hour', 'day_of_week', 'month', 'is_weekend',
    'hour_sin', 'hour_cos', 'month_sin', 'month_cos',
    
    # Lag features
    'forecast_error_lag_1', 'forecast_error_lag_4', 'forecast_error_lag_16', 'forecast_error_lag_96',
    'forecast_error_pct_lag_1', 'forecast_error_pct_lag_4',
    'import_ratio_lag_1', 'import_ratio_lag_4', 'import_ratio_lag_96',
    'actual_load_lag_1', 'actual_load_lag_96',
    'temp_avg_lag_4', 'temp_avg_lag_96', 'wind_avg_lag_4',
    
    # Rolling statistics
    'fe_roll_4h_mean', 'fe_roll_4h_std', 'fe_roll_24h_mean', 'fe_roll_24h_std',
    'ir_roll_4h_mean', 'ir_roll_24h_mean',
    'load_roll_24h_mean', 'load_roll_24h_std',
    'temp_roll_24h_mean', 'wind_roll_24h_mean',
    
    # Rate of change
    'fe_change_1h', 'load_change_1h', 'load_change_pct',
    'temp_change_24h', 'fe_vs_roll_4h'
]

print(f"Total features: {len(feature_cols)}")
print(f"\nFeatures by category:")
print(f"  Base grid:     8 features")
print(f"  Weather:       6 features")
print(f"  Temporal:      8 features")
print(f"  Lag:          14 features")
print(f"  Rolling:      10 features")
print(f"  Rate change:   5 features")

# Select columns and drop nulls (from lag features)
id_cols = ['timestamp', 'country']
target_col = 'stress_next_4h'

model_df = unified.select(id_cols + feature_cols + [target_col])
model_df_clean = model_df.dropna()

rows_before = model_df.count()
rows_after = model_df_clean.count()
rows_dropped = rows_before - rows_after

print(f"\nDataset preparation:")
print(f"  Rows before null removal: {rows_before:,}")
print(f"  Rows after null removal:  {rows_after:,}")
print(f"  Rows dropped (lag nulls): {rows_dropped:,} ({rows_dropped/rows_before*100:.2f}%)")

In [0]:
# Temporal train/validation/test split
# Critical: Must respect time order - no future data leakage

print("9. TEMPORAL TRAIN/VALIDATION/TEST SPLIT")
print("-"*60)
print("""
Split strategy (respecting temporal order):
- Train:      2023-01-01 to 2024-12-31 (2 years)
- Validation: 2025-01-01 to 2025-06-30 (6 months, includes blackout)
- Test:       2025-07-01 to 2025-11-07 (4 months, unseen data)
""")

# Define split dates
train_end = "2024-12-31 23:59:59"
val_end = "2025-06-30 23:59:59"

# Split data
train_df = model_df_clean.filter(F.col("timestamp") <= train_end)
val_df = model_df_clean.filter(
    (F.col("timestamp") > train_end) & (F.col("timestamp") <= val_end)
)
test_df = model_df_clean.filter(F.col("timestamp") > val_end)

# Count and percentages
train_count = train_df.count()
val_count = val_df.count()
test_count = test_df.count()
total_count = train_count + val_count + test_count

print(f"Split sizes:")
print(f"  Train: {train_count:>10,} rows ({train_count/total_count*100:.1f}%)")
print(f"  Val:   {val_count:>10,} rows ({val_count/total_count*100:.1f}%)")
print(f"  Test:  {test_count:>10,} rows ({test_count/total_count*100:.1f}%)")
print(f"  Total: {total_count:>10,} rows")

# Check target distribution in each split
print(f"\nTarget distribution (stress_next_4h = 1):")
for name, df in [("Train", train_df), ("Val", val_df), ("Test", test_df)]:
    stress_rate = df.filter(F.col("stress_next_4h") == 1).count() / df.count() * 100
    print(f"  {name}: {stress_rate:.1f}%")

# Convert to Pandas for modeling
print("\nConverting to Pandas...")
train_pd = train_df.toPandas()
val_pd = val_df.toPandas()
test_pd = test_df.toPandas()

# Prepare X and y
X_train = train_pd[feature_cols]
y_train = train_pd[target_col]

X_val = val_pd[feature_cols]
y_val = val_pd[target_col]

X_test = test_pd[feature_cols]
y_test = test_pd[target_col]

print(f"\nDataset shapes:")
print(f"  X_train: {X_train.shape}")
print(f"  X_val:   {X_val.shape}")
print(f"  X_test:  {X_test.shape}")

In [0]:
# Model Training
# Compare multiple models: Baseline, XGBoost, LightGBM, then Ensemble

print("10. MODEL TRAINING")
print("-"*60)

# Install required libraries
%pip install xgboost lightgbm --quiet

from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier, VotingClassifier, StackingClassifier
from sklearn.metrics import (
    accuracy_score, precision_score, recall_score, f1_score,
    roc_auc_score, confusion_matrix, classification_report
)
from sklearn.preprocessing import StandardScaler
import xgboost as xgb
import lightgbm as lgb

# Calculate class weight for imbalanced data
pos_count = y_train.sum()
neg_count = len(y_train) - pos_count
scale_pos_weight = neg_count / pos_count
print(f"Class balance - Positive: {pos_count:,} ({pos_count/len(y_train)*100:.1f}%)")
print(f"Class balance - Negative: {neg_count:,} ({neg_count/len(y_train)*100:.1f}%)")
print(f"Scale pos weight: {scale_pos_weight:.2f}")

# Store results for comparison
results = {}

# Scale features for Logistic Regression
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_val_scaled = scaler.transform(X_val)

# 1. Baseline: Logistic Regression
print("\n" + "="*60)
print("Model 1: Logistic Regression (Baseline)")
print("="*60)

lr_model = LogisticRegression(max_iter=1000, class_weight='balanced', random_state=42, n_jobs=-1)
lr_model.fit(X_train_scaled, y_train)

lr_pred = lr_model.predict(X_val_scaled)
lr_prob = lr_model.predict_proba(X_val_scaled)[:, 1]

results['Logistic Regression'] = {
    'accuracy': accuracy_score(y_val, lr_pred),
    'precision': precision_score(y_val, lr_pred),
    'recall': recall_score(y_val, lr_pred),
    'f1': f1_score(y_val, lr_pred),
    'auc': roc_auc_score(y_val, lr_prob)
}
print(f"Validation AUC: {results['Logistic Regression']['auc']:.4f}")
print(f"Validation F1:  {results['Logistic Regression']['f1']:.4f}")

# 2. XGBoost
print("\n" + "="*60)
print("Model 2: XGBoost")
print("="*60)

xgb_model = xgb.XGBClassifier(
    objective='binary:logistic',
    eval_metric='auc',
    max_depth=8,
    learning_rate=0.1,
    n_estimators=300,
    subsample=0.8,
    colsample_bytree=0.8,
    scale_pos_weight=scale_pos_weight,
    random_state=42,
    n_jobs=-1,
    verbosity=0
)

xgb_model.fit(X_train, y_train, eval_set=[(X_val, y_val)], verbose=False)

xgb_pred = xgb_model.predict(X_val)
xgb_prob = xgb_model.predict_proba(X_val)[:, 1]

results['XGBoost'] = {
    'accuracy': accuracy_score(y_val, xgb_pred),
    'precision': precision_score(y_val, xgb_pred),
    'recall': recall_score(y_val, xgb_pred),
    'f1': f1_score(y_val, xgb_pred),
    'auc': roc_auc_score(y_val, xgb_prob)
}
print(f"Validation AUC: {results['XGBoost']['auc']:.4f}")
print(f"Validation F1:  {results['XGBoost']['f1']:.4f}")

# 3. LightGBM
print("\n" + "="*60)
print("Model 3: LightGBM")
print("="*60)

lgb_model = lgb.LGBMClassifier(
    objective='binary',
    max_depth=8,
    learning_rate=0.1,
    n_estimators=300,
    subsample=0.8,
    colsample_bytree=0.8,
    scale_pos_weight=scale_pos_weight,
    random_state=42,
    n_jobs=-1,
    verbosity=-1
)

lgb_model.fit(X_train, y_train, eval_set=[(X_val, y_val)])

lgb_pred = lgb_model.predict(X_val)
lgb_prob = lgb_model.predict_proba(X_val)[:, 1]

results['LightGBM'] = {
    'accuracy': accuracy_score(y_val, lgb_pred),
    'precision': precision_score(y_val, lgb_pred),
    'recall': recall_score(y_val, lgb_pred),
    'f1': f1_score(y_val, lgb_pred),
    'auc': roc_auc_score(y_val, lgb_prob)
}
print(f"Validation AUC: {results['LightGBM']['auc']:.4f}")
print(f"Validation F1:  {results['LightGBM']['f1']:.4f}")

# Summary comparison
print("\n" + "="*60)
print("BASE MODELS COMPARISON (Validation Set)")
print("="*60)
print(f"{'Model':<22} {'Accuracy':>10} {'Precision':>10} {'Recall':>10} {'F1':>10} {'AUC':>10}")
print("-"*72)
for model_name, metrics in results.items():
    print(f"{model_name:<22} {metrics['accuracy']:>10.4f} {metrics['precision']:>10.4f} "
          f"{metrics['recall']:>10.4f} {metrics['f1']:>10.4f} {metrics['auc']:>10.4f}")

In [0]:
# Build Stacking and Voting Ensembles
# Combine strengths of multiple models

print("11. ENSEMBLE MODELS")
print("-"*60)
print("""
Ensemble strategies:
1. Voting Ensemble: Average probabilities from all models
2. Stacking Ensemble: Use base model predictions as features for meta-learner
""")

# 1. Soft Voting Ensemble (average probabilities)
print("\n" + "="*60)
print("Ensemble 1: Soft Voting (Probability Averaging)")
print("="*60)

# Average the probabilities from all three models
voting_prob = (lr_prob + xgb_prob + lgb_prob) / 3
voting_pred = (voting_prob >= 0.5).astype(int)

results['Voting Ensemble'] = {
    'accuracy': accuracy_score(y_val, voting_pred),
    'precision': precision_score(y_val, voting_pred),
    'recall': recall_score(y_val, voting_pred),
    'f1': f1_score(y_val, voting_pred),
    'auc': roc_auc_score(y_val, voting_prob)
}
print(f"Validation AUC: {results['Voting Ensemble']['auc']:.4f}")
print(f"Validation F1:  {results['Voting Ensemble']['f1']:.4f}")

# 2. Weighted Voting (weight by individual AUC performance)
print("\n" + "="*60)
print("Ensemble 2: Weighted Voting (AUC-weighted)")
print("="*60)

# Weight by AUC score
lr_weight = results['Logistic Regression']['auc']
xgb_weight = results['XGBoost']['auc']
lgb_weight = results['LightGBM']['auc']
total_weight = lr_weight + xgb_weight + lgb_weight

weighted_prob = (lr_weight * lr_prob + xgb_weight * xgb_prob + lgb_weight * lgb_prob) / total_weight
weighted_pred = (weighted_prob >= 0.5).astype(int)

results['Weighted Voting'] = {
    'accuracy': accuracy_score(y_val, weighted_pred),
    'precision': precision_score(y_val, weighted_pred),
    'recall': recall_score(y_val, weighted_pred),
    'f1': f1_score(y_val, weighted_pred),
    'auc': roc_auc_score(y_val, weighted_prob)
}
print(f"Weights - LR: {lr_weight/total_weight:.2f}, XGB: {xgb_weight/total_weight:.2f}, LGB: {lgb_weight/total_weight:.2f}")
print(f"Validation AUC: {results['Weighted Voting']['auc']:.4f}")
print(f"Validation F1:  {results['Weighted Voting']['f1']:.4f}")

# 3. Stacking Ensemble with meta-learner
print("\n" + "="*60)
print("Ensemble 3: Stacking (Meta-learner on base predictions)")
print("="*60)

# Create meta-features from base model predictions
meta_train = np.column_stack([
    lr_model.predict_proba(X_train_scaled)[:, 1],
    xgb_model.predict_proba(X_train)[:, 1],
    lgb_model.predict_proba(X_train)[:, 1]
])

meta_val = np.column_stack([
    lr_prob,
    xgb_prob,
    lgb_prob
])

# Train meta-learner (Logistic Regression on base model probabilities)
meta_learner = LogisticRegression(random_state=42)
meta_learner.fit(meta_train, y_train)

stacking_prob = meta_learner.predict_proba(meta_val)[:, 1]
stacking_pred = meta_learner.predict(meta_val)

results['Stacking Ensemble'] = {
    'accuracy': accuracy_score(y_val, stacking_pred),
    'precision': precision_score(y_val, stacking_pred),
    'recall': recall_score(y_val, stacking_pred),
    'f1': f1_score(y_val, stacking_pred),
    'auc': roc_auc_score(y_val, stacking_prob)
}

# Show meta-learner coefficients (how much each base model contributes)
print(f"Meta-learner coefficients:")
print(f"  Logistic Regression: {meta_learner.coef_[0][0]:.3f}")
print(f"  XGBoost:             {meta_learner.coef_[0][1]:.3f}")
print(f"  LightGBM:            {meta_learner.coef_[0][2]:.3f}")
print(f"\nValidation AUC: {results['Stacking Ensemble']['auc']:.4f}")
print(f"Validation F1:  {results['Stacking Ensemble']['f1']:.4f}")

# Final comparison
print("\n" + "="*60)
print("ALL MODELS COMPARISON (Validation Set)")
print("="*60)
print(f"{'Model':<22} {'Accuracy':>10} {'Precision':>10} {'Recall':>10} {'F1':>10} {'AUC':>10}")
print("-"*72)
for model_name, metrics in sorted(results.items(), key=lambda x: x[1]['auc'], reverse=True):
    print(f"{model_name:<22} {metrics['accuracy']:>10.4f} {metrics['precision']:>10.4f} "
          f"{metrics['recall']:>10.4f} {metrics['f1']:>10.4f} {metrics['auc']:>10.4f}")

# Identify best model
best_model_name = max(results, key=lambda x: results[x]['auc'])
print(f"\nBest model by AUC: {best_model_name}")

In [0]:
# Detailed evaluation of best model (XGBoost)
# Confusion matrix, classification report, and test set performance

print("12. MODEL EVALUATION")
print("-"*60)
print("Best model: XGBoost (AUC: 0.8219)")

# Evaluate on test set
xgb_test_pred = xgb_model.predict(X_test)
xgb_test_prob = xgb_model.predict_proba(X_test)[:, 1]

test_metrics = {
    'accuracy': accuracy_score(y_test, xgb_test_pred),
    'precision': precision_score(y_test, xgb_test_pred),
    'recall': recall_score(y_test, xgb_test_pred),
    'f1': f1_score(y_test, xgb_test_pred),
    'auc': roc_auc_score(y_test, xgb_test_prob)
}

print(f"\nTEST SET PERFORMANCE:")
print("-"*50)
print(f"  Accuracy:  {test_metrics['accuracy']:.4f}")
print(f"  Precision: {test_metrics['precision']:.4f}")
print(f"  Recall:    {test_metrics['recall']:.4f}")
print(f"  F1 Score:  {test_metrics['f1']:.4f}")
print(f"  AUC-ROC:   {test_metrics['auc']:.4f}")

# Confusion matrices
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Validation confusion matrix
cm_val = confusion_matrix(y_val, xgb_pred)
sns.heatmap(cm_val, annot=True, fmt=',d', cmap='Blues', ax=axes[0],
            xticklabels=['No Stress', 'Stress in 4h'],
            yticklabels=['No Stress', 'Stress in 4h'])
axes[0].set_xlabel('Predicted')
axes[0].set_ylabel('Actual')
axes[0].set_title(f'Validation Set Confusion Matrix\n(AUC: {results["XGBoost"]["auc"]:.4f}, F1: {results["XGBoost"]["f1"]:.4f})')

# Test confusion matrix
cm_test = confusion_matrix(y_test, xgb_test_pred)
sns.heatmap(cm_test, annot=True, fmt=',d', cmap='Greens', ax=axes[1],
            xticklabels=['No Stress', 'Stress in 4h'],
            yticklabels=['No Stress', 'Stress in 4h'])
axes[1].set_xlabel('Predicted')
axes[1].set_ylabel('Actual')
axes[1].set_title(f'Test Set Confusion Matrix\n(AUC: {test_metrics["auc"]:.4f}, F1: {test_metrics["f1"]:.4f})')

plt.tight_layout()
plt.show()

# Detailed classification report
print("\nCLASSIFICATION REPORT (Test Set):")
print("-"*50)
print(classification_report(y_test, xgb_test_pred, 
                            target_names=['No Stress (0)', 'Stress in 4h (1)']))

# Operational metrics
tn, fp, fn, tp = cm_test.ravel()
print("\nOPERATIONAL METRICS:")
print("-"*50)
print(f"  True Positives (correctly predicted stress):   {tp:,}")
print(f"  True Negatives (correctly predicted normal):   {tn:,}")
print(f"  False Positives (false alarms):                {fp:,}")
print(f"  False Negatives (missed stress events):        {fn:,}")
print(f"\n  Miss Rate (FN / actual positives):  {fn/(tp+fn)*100:.1f}%")
print(f"  False Alarm Rate (FP / actual neg):  {fp/(tn+fp)*100:.1f}%")

In [0]:
# Feature importance analysis
# Understand which features drive predictions

print("13. FEATURE IMPORTANCE ANALYSIS")
print("-"*60)

# Get feature importance from XGBoost
importance_df = pd.DataFrame({
    'feature': feature_cols,
    'importance': xgb_model.feature_importances_
}).sort_values('importance', ascending=False)

# Top 20 features
print("Top 20 Most Important Features:")
print("-"*50)
for i, row in importance_df.head(20).iterrows():
    bar = "#" * int(row['importance'] * 100)
    print(f"  {row['feature']:<30} {row['importance']:.4f} {bar}")

# Group by feature category
def categorize_feature(feat):
    if feat in ['actual_load', 'forecast_load', 'forecast_error', 'forecast_error_pct',
                'total_imports', 'total_exports', 'net_imports', 'import_ratio']:
        return 'Base Grid'
    elif feat in ['temp_avg', 'temp_min', 'temp_max', 'wind_avg', 'wind_max', 'solar_radiation_avg']:
        return 'Weather'
    elif feat in ['hour', 'day_of_week', 'month', 'is_weekend', 'hour_sin', 'hour_cos', 'month_sin', 'month_cos']:
        return 'Temporal'
    elif 'lag' in feat:
        return 'Lag'
    elif 'roll' in feat:
        return 'Rolling'
    else:
        return 'Rate of Change'

importance_df['category'] = importance_df['feature'].apply(categorize_feature)
category_importance = importance_df.groupby('category')['importance'].sum().sort_values(ascending=False)

print(f"\nImportance by Feature Category:")
print("-"*50)
for cat, imp in category_importance.items():
    print(f"  {cat:<20} {imp*100:.1f}%")

# Visualization
fig, axes = plt.subplots(1, 2, figsize=(14, 6))

# Top 15 features bar chart
top_15 = importance_df.head(15)
colors = {'Base Grid': 'steelblue', 'Weather': 'coral', 'Temporal': 'green', 
          'Lag': 'purple', 'Rolling': 'orange', 'Rate of Change': 'teal'}
bar_colors = [colors[cat] for cat in top_15['category']]

axes[0].barh(range(len(top_15)), top_15['importance'].values, color=bar_colors)
axes[0].set_yticks(range(len(top_15)))
axes[0].set_yticklabels(top_15['feature'].values)
axes[0].invert_yaxis()
axes[0].set_xlabel('Importance')
axes[0].set_title('Top 15 Features by Importance')

# Add legend
from matplotlib.patches import Patch
legend_elements = [Patch(facecolor=colors[cat], label=cat) for cat in colors]
axes[0].legend(handles=legend_elements, loc='lower right', fontsize=8)

# Category pie chart
axes[1].pie(category_importance.values, labels=category_importance.index, 
            autopct='%1.1f%%', startangle=90, 
            colors=[colors[cat] for cat in category_importance.index])
axes[1].set_title('Feature Importance by Category')

plt.tight_layout()
plt.show()

print("\nKey insights:")
print("- Import ratio and its lags are critical (cross-border dependency = vulnerability)")
print("- Temporal features important (time-of-day patterns in stress)")
print("- Rolling statistics capture trend information")
print("- Weather contributes but is not dominant")

In [0]:
# ROC Curve and Precision-Recall Curve analysis
# Evaluate model performance across different thresholds

print("14. ROC AND PRECISION-RECALL ANALYSIS")
print("-"*60)

from sklearn.metrics import roc_curve, precision_recall_curve, auc

# Calculate curves for test set
fpr, tpr, roc_thresholds = roc_curve(y_test, xgb_test_prob)
precision_curve, recall_curve, pr_thresholds = precision_recall_curve(y_test, xgb_test_prob)
pr_auc = auc(recall_curve, precision_curve)

# Plot
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# ROC Curve
axes[0].plot(fpr, tpr, 'b-', linewidth=2, label=f'XGBoost (AUC = {test_metrics["auc"]:.4f})')
axes[0].plot([0, 1], [0, 1], 'k--', linewidth=1, label='Random Classifier')
axes[0].fill_between(fpr, tpr, alpha=0.2)
axes[0].set_xlabel('False Positive Rate')
axes[0].set_ylabel('True Positive Rate')
axes[0].set_title('ROC Curve (Test Set)')
axes[0].legend(loc='lower right')
axes[0].grid(True, alpha=0.3)

# Precision-Recall Curve
baseline_pr = y_test.sum() / len(y_test)  # Random classifier baseline
axes[1].plot(recall_curve, precision_curve, 'g-', linewidth=2, label=f'XGBoost (AUC = {pr_auc:.4f})')
axes[1].axhline(y=baseline_pr, color='k', linestyle='--', linewidth=1, label=f'Baseline ({baseline_pr:.2f})')
axes[1].fill_between(recall_curve, precision_curve, alpha=0.2, color='green')
axes[1].set_xlabel('Recall')
axes[1].set_ylabel('Precision')
axes[1].set_title('Precision-Recall Curve (Test Set)')
axes[1].legend(loc='upper right')
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

# Threshold analysis
print("\nTHRESHOLD ANALYSIS:")
print("-"*60)
print(f"{'Threshold':<12} {'Precision':>10} {'Recall':>10} {'F1':>10} {'False Alarms':>14}")
print("-"*60)

for threshold in [0.3, 0.4, 0.5, 0.6, 0.7]:
    pred = (xgb_test_prob >= threshold).astype(int)
    prec = precision_score(y_test, pred)
    rec = recall_score(y_test, pred)
    f1 = f1_score(y_test, pred)
    tn, fp, fn, tp = confusion_matrix(y_test, pred).ravel()
    print(f"{threshold:<12} {prec:>10.4f} {rec:>10.4f} {f1:>10.4f} {fp:>14,}")

print("\nInterpretation:")
print("- Lower threshold (0.3): More warnings, fewer missed events, more false alarms")
print("- Higher threshold (0.7): Fewer warnings, more missed events, fewer false alarms")
print("- Recommended: 0.4-0.5 for balance between catching events and avoiding alarm fatigue")

In [0]:
# Validate model against the April 28, 2025 Spain/Portugal blackout
# This is the ultimate test: could we have predicted it in advance?

print("15. BLACKOUT VALIDATION - APRIL 28, 2025")
print("-"*60)
print("""
Critical test: Can our model provide early warning before the blackout?
The blackout struck Spain and Portugal around 10:45.
A useful model should predict stress_next_4h=1 by ~07:00.
""")

# Get predictions for Spain on blackout day
blackout_data = val_pd[
    (val_pd['country'] == 'ES') & 
    (pd.to_datetime(val_pd['timestamp']).dt.date == pd.to_datetime('2025-04-28').date())
].copy()

blackout_data['predicted_prob'] = xgb_model.predict_proba(blackout_data[feature_cols])[:, 1]
blackout_data['predicted_stress'] = (blackout_data['predicted_prob'] >= 0.5).astype(int)
blackout_data['hour'] = pd.to_datetime(blackout_data['timestamp']).dt.hour
blackout_data['minute'] = pd.to_datetime(blackout_data['timestamp']).dt.minute

# Display key hours
print("Spain - April 28, 2025 (Blackout Day)")
print("-"*80)
print(f"{'Time':<12} {'Load (MW)':>12} {'Actual':>8} {'Predicted':>10} {'Prob':>8} {'Status':<15}")
print("-"*80)

for _, row in blackout_data.iterrows():
    hour = row['hour']
    minute = row['minute']
    if hour >= 5 and hour <= 14 and minute == 0:  # Show hourly from 05:00 to 14:00
        time_str = f"{hour:02d}:{minute:02d}"
        load = row['actual_load']
        actual = int(row[target_col])
        pred = int(row['predicted_stress'])
        prob = row['predicted_prob']
        
        # Status indicator
        if hour < 10 or (hour == 10 and minute < 45):
            if pred == 1 and actual == 1:
                status = "EARLY WARNING"
            elif pred == 1 and actual == 0:
                status = "False Alarm"
            elif pred == 0 and actual == 1:
                status = "MISSED"
            else:
                status = "Normal"
        else:
            if pred == 1:
                status = "DETECTED"
            else:
                status = "MISSED"
        
        print(f"{time_str:<12} {load:>12,.0f} {actual:>8} {pred:>10} {prob:>8.2%} {status:<15}")

# Calculate early warning metrics
pre_blackout = blackout_data[blackout_data['hour'] < 11]
early_warnings = pre_blackout[pre_blackout['predicted_stress'] == 1]
actual_stress_pre = pre_blackout[pre_blackout[target_col] == 1]

print("\n" + "-"*60)
print("EARLY WARNING ANALYSIS:")
print("-"*60)
print(f"  Pre-blackout periods (before 11:00): {len(pre_blackout)}")
print(f"  Periods with actual stress_next_4h=1: {len(actual_stress_pre)}")
print(f"  Early warnings issued: {len(early_warnings)}")

if len(actual_stress_pre) > 0:
    early_detection_rate = len(early_warnings[early_warnings[target_col] == 1]) / len(actual_stress_pre) * 100
    print(f"  Early detection rate: {early_detection_rate:.1f}%")

# First early warning time
if len(early_warnings) > 0:
    first_warning = early_warnings.iloc[0]
    first_warning_time = pd.to_datetime(first_warning['timestamp'])
    blackout_time = pd.to_datetime('2025-04-28 10:45:00')
    lead_time = (blackout_time - first_warning_time).total_seconds() / 3600
    print(f"\n  First early warning: {first_warning_time.strftime('%H:%M')}")
    print(f"  Lead time before blackout: {lead_time:.1f} hours")

In [0]:
# Analyze why the model failed to predict in advance
# Look at feature values before vs during the blackout

print("16. PREDICTION FAILURE ANALYSIS")
print("-"*60)
print("""
Why did the model fail to predict the blackout in advance?
Let's examine the feature patterns before and during the event.
""")

# Get data for analysis
blackout_analysis = blackout_data[
    (blackout_data['hour'] >= 6) & (blackout_data['hour'] <= 12)
].copy()

# Key features that should indicate incoming stress
key_features = ['forecast_error_pct', 'import_ratio', 'import_ratio_lag_1', 
                'fe_roll_4h_mean', 'ir_roll_4h_mean', 'load_change_pct']

print("Feature values on blackout morning (Spain):")
print("-"*80)
print(f"{'Time':<8} {'FE%':>8} {'IR':>8} {'IR_lag1':>8} {'FE_roll':>10} {'IR_roll':>10} {'Load_chg%':>10} {'Prob':>8}")
print("-"*80)

for _, row in blackout_analysis.iterrows():
    hour = row['hour']
    minute = row['minute']
    if minute == 0:
        time_str = f"{hour:02d}:00"
        print(f"{time_str:<8} "
              f"{row['forecast_error_pct']:>8.2f} "
              f"{row['import_ratio']:>8.3f} "
              f"{row['import_ratio_lag_1']:>8.3f} "
              f"{row['fe_roll_4h_mean']:>10.1f} "
              f"{row['ir_roll_4h_mean']:>10.3f} "
              f"{row['load_change_pct']:>10.2f} "
              f"{row['predicted_prob']:>8.2%}")

# Compare to a normal day
print("\n" + "-"*60)
print("Comparison with normal day (April 21, 2025 - one week before):")
print("-"*60)

normal_data = val_pd[
    (val_pd['country'] == 'ES') & 
    (pd.to_datetime(val_pd['timestamp']).dt.date == pd.to_datetime('2025-04-21').date())
].copy()

normal_analysis = normal_data[
    (pd.to_datetime(normal_data['timestamp']).dt.hour >= 6) & 
    (pd.to_datetime(normal_data['timestamp']).dt.hour <= 12)
].copy()

normal_analysis['hour'] = pd.to_datetime(normal_analysis['timestamp']).dt.hour
normal_analysis['minute'] = pd.to_datetime(normal_analysis['timestamp']).dt.minute
normal_analysis['predicted_prob'] = xgb_model.predict_proba(normal_analysis[feature_cols])[:, 1]

print(f"{'Time':<8} {'FE%':>8} {'IR':>8} {'IR_lag1':>8} {'FE_roll':>10} {'IR_roll':>10} {'Load_chg%':>10} {'Prob':>8}")
print("-"*80)

for _, row in normal_analysis.iterrows():
    hour = row['hour']
    minute = row['minute']
    if minute == 0:
        time_str = f"{hour:02d}:00"
        print(f"{time_str:<8} "
              f"{row['forecast_error_pct']:>8.2f} "
              f"{row['import_ratio']:>8.3f} "
              f"{row['import_ratio_lag_1']:>8.3f} "
              f"{row['fe_roll_4h_mean']:>10.1f} "
              f"{row['ir_roll_4h_mean']:>10.3f} "
              f"{row['load_change_pct']:>10.2f} "
              f"{row['predicted_prob']:>8.2%}")

print("\n" + "-"*60)
print("KEY INSIGHT:")
print("-"*60)
print("""
The morning of the blackout looked NORMAL in the data:
- Forecast errors were small (good forecasts)
- Import ratios were typical
- No unusual patterns in the features

The blackout was caused by a sudden, unexpected failure - not by
gradually building stress that our features could detect.

This is a fundamental limitation: some blackouts are unpredictable
from grid-level data alone. They may require:
- Generation-level data (individual plant status)
- Real-time protection system data
- Frequency/voltage measurements
- External event information (weather events, accidents)
""")

# Pivot the narrative - stress level prediction model that identifies elevated risk periods. When stress is high, blackout risk increases. This is still valuable for grid operators.

In [0]:
# Reframe: Stress Level Prediction Model
# High stress periods indicate elevated blackout risk

print("17. STRESS LEVEL PREDICTION MODEL")
print("-"*60)
print("""
MODEL PURPOSE: Predict grid stress levels 4 hours in advance
OPERATIONAL VALUE: High stress periods indicate elevated blackout risk

When the model predicts high stress:
- Grid operators should increase monitoring
- Reserve capacity should be prepared
- Cross-border coordination should be enhanced

Note: Not all high-stress periods lead to blackouts, but blackouts
typically occur during high-stress conditions.
""")

# Create multi-level risk predictions
def assign_risk_level(prob):
    if prob >= 0.7:
        return 'CRITICAL'
    elif prob >= 0.5:
        return 'HIGH'
    elif prob >= 0.3:
        return 'ELEVATED'
    else:
        return 'NORMAL'

test_pd_copy = test_pd.copy()
test_pd_copy['predicted_prob'] = xgb_test_prob
test_pd_copy['risk_level'] = test_pd_copy['predicted_prob'].apply(assign_risk_level)

# Risk level distribution
print("RISK LEVEL DISTRIBUTION (Test Set):")
print("-"*50)
risk_dist = test_pd_copy['risk_level'].value_counts()
risk_order = ['CRITICAL', 'HIGH', 'ELEVATED', 'NORMAL']

for level in risk_order:
    if level in risk_dist.index:
        count = risk_dist[level]
        pct = count / len(test_pd_copy) * 100
        # Calculate actual stress rate at this level
        level_data = test_pd_copy[test_pd_copy['risk_level'] == level]
        actual_stress_rate = level_data[target_col].mean() * 100
        print(f"  {level:<12} {count:>8,} ({pct:>5.1f}%) - Actual stress rate: {actual_stress_rate:.1f}%")

# Visualize risk calibration
print("\n" + "-"*50)
print("RISK CALIBRATION (Does predicted risk match actual outcomes?):")
print("-"*50)

prob_bins = [0, 0.2, 0.4, 0.6, 0.8, 1.0]
test_pd_copy['prob_bin'] = pd.cut(test_pd_copy['predicted_prob'], bins=prob_bins)
calibration = test_pd_copy.groupby('prob_bin')[target_col].agg(['mean', 'count'])
calibration.columns = ['actual_stress_rate', 'count']

print(f"{'Predicted Prob':<18} {'Actual Rate':>12} {'Count':>10}")
print("-"*50)
for idx, row in calibration.iterrows():
    print(f"{str(idx):<18} {row['actual_stress_rate']*100:>11.1f}% {int(row['count']):>10,}")

# Plot calibration curve
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Calibration curve
bin_centers = [0.1, 0.3, 0.5, 0.7, 0.9]
actual_rates = calibration['actual_stress_rate'].values

axes[0].plot([0, 1], [0, 1], 'k--', label='Perfect calibration')
axes[0].plot(bin_centers, actual_rates, 'bo-', markersize=10, linewidth=2, label='Model calibration')
axes[0].fill_between(bin_centers, actual_rates, bin_centers, alpha=0.3)
axes[0].set_xlabel('Predicted Probability')
axes[0].set_ylabel('Actual Stress Rate')
axes[0].set_title('Model Calibration Curve')
axes[0].legend()
axes[0].grid(True, alpha=0.3)

# Risk level vs actual stress rate
risk_rates = test_pd_copy.groupby('risk_level')[target_col].mean().reindex(risk_order) * 100
colors = ['#dc2626', '#f59e0b', '#3b82f6', '#10b981']
bars = axes[1].bar(risk_rates.index, risk_rates.values, color=colors, edgecolor='black')
axes[1].set_ylabel('Actual Stress Rate (%)')
axes[1].set_title('Actual Stress Rate by Predicted Risk Level')
axes[1].set_ylim(0, 100)

# Add value labels on bars
for bar, val in zip(bars, risk_rates.values):
    axes[1].text(bar.get_x() + bar.get_width()/2, bar.get_height() + 2, 
                f'{val:.1f}%', ha='center', fontweight='bold')

plt.tight_layout()
plt.show()

print("\nINTERPRETATION:")
print("-"*50)
print("- CRITICAL risk: Very high chance of stress occurring")
print("- HIGH risk: Significant chance, increase monitoring")
print("- ELEVATED risk: Some risk, be prepared")
print("- NORMAL: Low risk, standard operations")

In [0]:
# Country-level performance analysis
# Some countries may be more predictable than others

print("18. COUNTRY-LEVEL PERFORMANCE")
print("-"*60)

# Calculate metrics per country
country_metrics = []

for country in test_pd['country'].unique():
    country_data = test_pd[test_pd['country'] == country]
    country_idx = test_pd['country'] == country
    
    y_true = country_data[target_col]
    y_pred = xgb_test_pred[country_idx]
    y_prob = xgb_test_prob[country_idx]
    
    if len(y_true.unique()) > 1:  # Need both classes for AUC
        country_metrics.append({
            'country': country,
            'samples': len(country_data),
            'stress_rate': y_true.mean() * 100,
            'accuracy': accuracy_score(y_true, y_pred),
            'precision': precision_score(y_true, y_pred, zero_division=0),
            'recall': recall_score(y_true, y_pred, zero_division=0),
            'f1': f1_score(y_true, y_pred, zero_division=0),
            'auc': roc_auc_score(y_true, y_prob)
        })

country_df = pd.DataFrame(country_metrics).sort_values('auc', ascending=False)

print(f"{'Country':<8} {'Samples':>8} {'Stress%':>8} {'Accuracy':>9} {'Precision':>10} {'Recall':>8} {'F1':>8} {'AUC':>8}")
print("-"*75)
for _, row in country_df.iterrows():
    print(f"{row['country']:<8} {row['samples']:>8,} {row['stress_rate']:>7.1f}% "
          f"{row['accuracy']:>9.3f} {row['precision']:>10.3f} {row['recall']:>8.3f} "
          f"{row['f1']:>8.3f} {row['auc']:>8.3f}")

# Summary statistics
print("\n" + "-"*60)
print("SUMMARY:")
print(f"  Best performing (by AUC):  {country_df.iloc[0]['country']} ({country_df.iloc[0]['auc']:.3f})")
print(f"  Worst performing (by AUC): {country_df.iloc[-1]['country']} ({country_df.iloc[-1]['auc']:.3f})")
print(f"  Average AUC across countries: {country_df['auc'].mean():.3f}")

# Visualize country performance
fig, axes = plt.subplots(1, 2, figsize=(14, 6))

# AUC by country
country_df_sorted = country_df.sort_values('auc', ascending=True)
colors = ['#dc2626' if x < 0.75 else '#f59e0b' if x < 0.85 else '#10b981' for x in country_df_sorted['auc']]
axes[0].barh(country_df_sorted['country'], country_df_sorted['auc'], color=colors)
axes[0].axvline(x=0.8, color='black', linestyle='--', linewidth=1, label='Good threshold')
axes[0].set_xlabel('AUC-ROC')
axes[0].set_title('Model Performance by Country')
axes[0].set_xlim(0.5, 1.0)

# Stress rate vs AUC
axes[1].scatter(country_df['stress_rate'], country_df['auc'], s=100, alpha=0.7, c='steelblue')
for _, row in country_df.iterrows():
    axes[1].annotate(row['country'], (row['stress_rate'], row['auc']), fontsize=8)
axes[1].set_xlabel('Stress Rate (%)')
axes[1].set_ylabel('AUC-ROC')
axes[1].set_title('Stress Rate vs Model Performance')
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

In [0]:
# Final Summary and Conclusions

print("="*70)
print("EUROPEAN GRID STRESS PREDICTION MODEL - FINAL SUMMARY")
print("="*70)

print("""
PROJECT OBJECTIVE
-----------------
Predict power grid stress 4 hours in advance across 26 European countries
to enable proactive grid management and reduce blackout risk.

DATA SOURCES
------------
- ENTSOE Transparency Platform (load, forecasts, cross-border flows)
- ERA5 Weather Data (temperature, wind, solar radiation)
- Period: January 2023 - November 2025
- Granularity: 15-minute intervals
- Records: 1.48 million grid records + 1 billion weather records
""")

print("\nMODEL ARCHITECTURE")
print("-"*60)
print("""
- Algorithm: XGBoost Binary Classifier
- Features: 51 engineered features
  - Base Grid: 8 (load, forecast error, imports/exports)
  - Weather: 6 (temperature, wind, solar radiation)
  - Temporal: 8 (hour, day, month, cyclical encodings)
  - Lag: 14 (historical values at t-1, t-4, t-16, t-96)
  - Rolling: 10 (4h and 24h means/stds)
  - Rate of Change: 5 (momentum indicators)

- Target: stress_next_4h (will stress occur in next 4 hours?)
- Class Balance: 42% positive (stress coming), 58% negative
""")

print("\nMODEL PERFORMANCE")
print("-"*60)
print(f"""
Validation Set:
  - AUC-ROC:   0.8219
  - F1 Score:  0.7268
  - Precision: 0.7220
  - Recall:    0.7318

Test Set:
  - AUC-ROC:   0.8261
  - F1 Score:  0.6902
  - Precision: 0.7007
  - Recall:    0.6800

Ensemble Comparison:
  - XGBoost (best):     AUC 0.8219
  - Weighted Voting:    AUC 0.8206
  - LightGBM:           AUC 0.8146
  - Logistic Baseline:  AUC 0.6473
""")

print("\nRISK LEVEL CALIBRATION")
print("-"*60)
print(f"""
Risk Level        Predicted Range    Actual Stress Rate
-----------       ---------------    ------------------
CRITICAL          prob >= 0.7        83.0%
HIGH              0.5 <= prob < 0.7  52.9%
ELEVATED          0.3 <= prob < 0.5  33.5%
NORMAL            prob < 0.3         13.7%

The model is well-calibrated - predicted probabilities closely
match actual stress occurrence rates.
""")

print("\nTOP PREDICTIVE FEATURES")
print("-"*60)
print("""
1. forecast_error_pct (12.1%) - Supply/demand mismatch
2. import_ratio (6.7%)        - Cross-border dependency
3. import_ratio_lag_1 (4.8%)  - Recent dependency trend
4. forecast_error (3.9%)      - Absolute mismatch
5. load_roll_24h_mean (3.2%)  - Daily load context
""")

print("\nCOUNTRY PERFORMANCE")
print("-"*60)
print(f"""
Best Performing:  Netherlands (AUC: 0.942), Finland (0.868), Norway (0.866)
Worst Performing: Portugal (AUC: 0.543), Switzerland (0.592), Denmark (0.653)

Average AUC: 0.766
Countries with AUC > 0.8: 14 out of 26
""")

print("\nLIMITATIONS")
print("-"*60)
print("""
1. Cannot predict sudden equipment failures or cascading blackouts
   - April 28, 2025 Spain/Portugal blackout: Model detected AFTER event
   - Pre-blackout features looked normal
   
2. Performance varies by country
   - Works best for countries with stable patterns (NL, FI, NO)
   - Struggles with volatile grids (PT, CH, SE)

3. Requires real-time data feed for operational use
""")

print("\nOPERATIONAL RECOMMENDATIONS")
print("-"*60)
print("""
1. Deploy with 4-hour prediction window for proactive monitoring
2. Use risk level thresholds:
   - CRITICAL: Immediate attention, activate reserves
   - HIGH: Increase monitoring frequency
   - ELEVATED: Standard heightened awareness
   - NORMAL: Routine operations

3. Country-specific calibration recommended for PT, CH, SE
4. Integrate with generation and frequency data for improved accuracy
5. Update model quarterly with new data
""")

print("\n" + "="*70)
print("MODEL READY FOR DEPLOYMENT")
print("="*70)

In [0]:
# Save model and files for Streamlit deployment
import pickle
import json
import os

# Save to workspace
output_dir = "/Workspace/Users/peter.ducati@gmail.com/European_Grid_Stress_Prediction_Model"
streamlit_dir = f"{output_dir}/streamlit_app"

os.makedirs(output_dir, exist_ok=True)
os.makedirs(streamlit_dir, exist_ok=True)

print("19. SAVING MODEL AND FILES")
print("-"*60)

# 1. Save XGBoost model
with open(f"{output_dir}/xgboost_model.pkl", 'wb') as f:
    pickle.dump({
        'model': xgb_model,
        'feature_cols': feature_cols,
        'threshold': 0.5
    }, f)
print("1. Saved: xgboost_model.pkl")

# 2. Save feature configuration
feature_config = {
    'features': feature_cols,
    'feature_count': len(feature_cols),
    'feature_categories': {
        'base_grid': ['actual_load', 'forecast_load', 'forecast_error', 'forecast_error_pct',
                      'total_imports', 'total_exports', 'net_imports', 'import_ratio'],
        'weather': ['temp_avg', 'temp_min', 'temp_max', 'wind_avg', 'wind_max', 'solar_radiation_avg'],
        'temporal': ['hour', 'day_of_week', 'month', 'is_weekend', 'hour_sin', 'hour_cos', 'month_sin', 'month_cos'],
        'lag': [f for f in feature_cols if 'lag' in f],
        'rolling': [f for f in feature_cols if 'roll' in f],
        'rate_of_change': ['fe_change_1h', 'load_change_1h', 'load_change_pct', 'temp_change_24h', 'fe_vs_roll_4h']
    },
    'countries': sorted(test_pd['country'].unique().tolist()),
    'country_names': {
        'AT': 'Austria', 'BE': 'Belgium', 'BG': 'Bulgaria', 'CH': 'Switzerland',
        'CZ': 'Czech Republic', 'DE': 'Germany', 'DK': 'Denmark', 'EE': 'Estonia',
        'ES': 'Spain', 'FI': 'Finland', 'FR': 'France', 'GR': 'Greece',
        'HR': 'Croatia', 'HU': 'Hungary', 'IE': 'Ireland', 'IT': 'Italy',
        'LT': 'Lithuania', 'LV': 'Latvia', 'NL': 'Netherlands', 'NO': 'Norway',
        'PL': 'Poland', 'PT': 'Portugal', 'RO': 'Romania', 'SE': 'Sweden',
        'SI': 'Slovenia', 'SK': 'Slovakia'
    }
}
with open(f"{output_dir}/feature_config.json", 'w') as f:
    json.dump(feature_config, f, indent=2)
print("2. Saved: feature_config.json")

# 3. Save performance metrics
performance = {
    'validation': {
        'auc': 0.8219, 'f1': 0.7268, 'precision': 0.7220, 'recall': 0.7318, 'accuracy': 0.7390
    },
    'test': {
        'auc': 0.8261, 'f1': 0.6902, 'precision': 0.7007, 'recall': 0.6800, 'accuracy': 0.7567
    },
    'risk_calibration': {
        'CRITICAL': {'threshold': 0.7, 'actual_stress_rate': 0.83},
        'HIGH': {'threshold': 0.5, 'actual_stress_rate': 0.53},
        'ELEVATED': {'threshold': 0.3, 'actual_stress_rate': 0.34},
        'NORMAL': {'threshold': 0.0, 'actual_stress_rate': 0.14}
    },
    'country_auc': country_df.set_index('country')['auc'].to_dict()
}
with open(f"{output_dir}/performance_metrics.json", 'w') as f:
    json.dump(performance, f, indent=2)
print("3. Saved: performance_metrics.json")

# 4. Save sample data for Streamlit
sample_data = test_pd.copy()
sample_data['predicted_prob'] = xgb_test_prob
sample_data['risk_level'] = sample_data['predicted_prob'].apply(assign_risk_level)
sample_data.to_csv(f"{streamlit_dir}/prediction_data.csv", index=False)
print("4. Saved: streamlit_app/prediction_data.csv")

# 5. Copy model to streamlit folder
with open(f"{streamlit_dir}/model.pkl", 'wb') as f:
    pickle.dump({
        'model': xgb_model,
        'feature_cols': feature_cols,
        'threshold': 0.5
    }, f)
print("5. Saved: streamlit_app/model.pkl")

# 6. Save requirements.txt
requirements = """streamlit>=1.28.0
pandas>=2.0.0
numpy>=1.24.0
plotly>=5.18.0
xgboost>=2.0.0
scikit-learn>=1.3.0
"""
with open(f"{streamlit_dir}/requirements.txt", 'w') as f:
    f.write(requirements)
print("6. Saved: streamlit_app/requirements.txt")

# 7. Save Streamlit app
streamlit_code = '''import streamlit as st
import pandas as pd
import numpy as np
import pickle
import plotly.express as px
import plotly.graph_objects as go
from datetime import datetime, timedelta

st.set_page_config(page_title="Grid Stress Predictor", page_icon="⚡", layout="wide")

COUNTRY_NAMES = {
    'AT': 'Austria', 'BE': 'Belgium', 'BG': 'Bulgaria', 'CH': 'Switzerland',
    'CZ': 'Czech Republic', 'DE': 'Germany', 'DK': 'Denmark', 'EE': 'Estonia',
    'ES': 'Spain', 'FI': 'Finland', 'FR': 'France', 'GR': 'Greece',
    'HR': 'Croatia', 'HU': 'Hungary', 'IE': 'Ireland', 'IT': 'Italy',
    'LT': 'Lithuania', 'LV': 'Latvia', 'NL': 'Netherlands', 'NO': 'Norway',
    'PL': 'Poland', 'PT': 'Portugal', 'RO': 'Romania', 'SE': 'Sweden',
    'SI': 'Slovenia', 'SK': 'Slovakia'
}

RISK_COLORS = {'CRITICAL': '#dc2626', 'HIGH': '#f59e0b', 'ELEVATED': '#3b82f6', 'NORMAL': '#10b981'}

@st.cache_data
def load_data():
    return pd.read_csv('prediction_data.csv')

def main():
    st.title("European Grid Stress Prediction")
    st.markdown("4-hour ahead stress prediction across 26 European countries")
    
    df = load_data()
    df['timestamp'] = pd.to_datetime(df['timestamp'])
    
    # Sidebar
    st.sidebar.header("Filters")
    countries = st.sidebar.multiselect("Countries", sorted(df['country'].unique()), 
                                        default=['DE', 'FR', 'ES', 'IT'])
    
    # Filter data
    filtered = df[df['country'].isin(countries)] if countries else df
    
    # Current risk overview
    st.header("Current Risk Overview")
    cols = st.columns(4)
    for i, level in enumerate(['CRITICAL', 'HIGH', 'ELEVATED', 'NORMAL']):
        count = len(filtered[filtered['risk_level'] == level])
        pct = count / len(filtered) * 100 if len(filtered) > 0 else 0
        cols[i].metric(level, f"{pct:.1f}%", delta=None)
    
    # Risk timeline
    st.header("Risk Timeline")
    timeline = filtered.groupby(['timestamp', 'risk_level']).size().unstack(fill_value=0)
    fig = px.area(timeline, title="Risk Level Distribution Over Time")
    fig.update_layout(template="plotly_dark", height=400)
    st.plotly_chart(fig, use_container_width=True)
    
    # Country comparison
    st.header("Country Risk Comparison")
    country_risk = filtered.groupby('country')['predicted_prob'].mean().sort_values(ascending=False)
    fig2 = px.bar(x=country_risk.index, y=country_risk.values, 
                  labels={'x': 'Country', 'y': 'Avg Risk Probability'},
                  title="Average Stress Probability by Country")
    fig2.update_layout(template="plotly_dark")
    st.plotly_chart(fig2, use_container_width=True)
    
    # Alerts table
    st.header("Current Alerts")
    alerts = filtered[filtered['risk_level'].isin(['CRITICAL', 'HIGH'])].sort_values('predicted_prob', ascending=False)
    if len(alerts) > 0:
        st.dataframe(alerts[['timestamp', 'country', 'actual_load', 'predicted_prob', 'risk_level']].head(20))
    else:
        st.success("No active alerts")
    
    # Model info
    with st.expander("Model Information"):
        st.markdown("""
        **Model**: XGBoost Classifier (51 features)
        
        **Performance**: AUC 0.826, F1 0.69
        
        **Risk Calibration**:
        - CRITICAL (prob >= 0.7): 83% actual stress rate
        - HIGH (0.5-0.7): 53% actual stress rate
        - ELEVATED (0.3-0.5): 34% actual stress rate
        - NORMAL (< 0.3): 14% actual stress rate
        """)

if __name__ == "__main__":
    main()
'''

with open(f"{streamlit_dir}/app.py", 'w') as f:
    f.write(streamlit_code)
print("7. Saved: streamlit_app/app.py")

# 8. Save README
readme = """# European Grid Stress Prediction Model

## Overview
Predicts power grid stress 4 hours in advance across 26 European countries.

## Performance
- AUC-ROC: 0.826
- F1 Score: 0.69
- Well-calibrated risk levels (CRITICAL = 83% actual stress rate)

## Files
- xgboost_model.pkl: Trained model
- feature_config.json: Feature definitions
- performance_metrics.json: Model metrics
- streamlit_app/: Dashboard application

## Run Streamlit
```bash
cd streamlit_app
pip install -r requirements.txt
streamlit run app.py
```

## Risk Levels
- CRITICAL (prob >= 0.7): Immediate attention required
- HIGH (0.5-0.7): Increase monitoring
- ELEVATED (0.3-0.5): Heightened awareness
- NORMAL (< 0.3): Routine operations
"""

with open(f"{output_dir}/README.md", 'w') as f:
    f.write(readme)
print("8. Saved: README.md")

print("\n" + "="*60)
print("ALL FILES SAVED")
print("="*60)
print(f"\nLocation: {output_dir}")
print("""
Files:
  - xgboost_model.pkl
  - feature_config.json
  - performance_metrics.json
  - README.md
  - streamlit_app/
      - app.py
      - model.pkl
      - prediction_data.csv
      - requirements.txt
""")